In [1]:
#Importing importing Files

import re
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_lg")

In [2]:
#Reading File Data

fileData=[]
with open("Input_file.txt", "r") as file:
    for line in file:
        x = re.sub("=", " = ", line)
        x = re.sub("  "," ", x)   
        fileData.append(x)

In [3]:
#Defining Patterns

matcher = Matcher(nlp.vocab)

pattern1 = [{"LOWER":"integer"}]
pattern2 = [{"LOWER":"int"}]
pattern3 = [{"LOWER":"float"}]
pattern4 = [{"LOWER":"number"}]
pattern5 = [{"LOWER":"num"}]
pattern6 = [{"LOWER":"variable"}]
pattern7 = [{"LOWER":"var"}]
pattern8 = [{"LOWER":"double"}]
pattern9 = [{"LOWER":"character"}]
pattern10 = [{"LOWER":"char"}]
pattern11 = [{"LOWER":"string"}]
pattern12 = [{"LOWER":"str"}]
pattern13 = [{"LOWER":"void"}]
pattern14 = [{"LOWER":"array"}]
pattern15 = [{"LOWER":"list"}]
pattern16 = [{"LOWER":"vector"}]
pattern17 = [{"LOWER":"class"}]
matcher.add("DATATYPE", None, pattern1, pattern2, pattern3, pattern4, pattern5, pattern6, pattern7, 
            pattern8, pattern9, pattern10, pattern11, pattern12, pattern13, pattern14, pattern15, 
            pattern16, pattern17)

pattern2_1 = [{"LOWER":"global"}]
pattern2_2 = [{"LOWER":"local"}]
matcher.add("SCOPE", None, pattern2_1, pattern2_2)

pattern3_1 = [{"LOWER":"addition"}]
pattern3_2 = [{"LOWER":"add"}]
pattern3_3 = [{"TEXT":"+"}]
pattern3_4 = [{"LOWER":"subtraction"}]
pattern3_5 = [{"LOWER":"sub"}]
pattern3_6 = [{"TEXT":"-"}]
pattern3_7 = [{"LOWER":"minus"}]
pattern3_8 = [{"LOWER":"multiply"}]
pattern3_9 = [{"LOWER":"mul"}]
pattern3_10 = [{"TEXT":"*"}]
pattern3_11 = [{"LOWER":"divide"}]
pattern3_12 = [{"LOWER":"div"}]
pattern3_13 = [{"TEXT":"/"}]
pattern3_14 = [{"LOWER":"modulo"}]
pattern3_15 = [{"LOWER":"modulus"}]
pattern3_16 = [{"LOWER":"mod"}]
pattern3_17 = [{"TEXT":"%"}]
matcher.add("OPERATOR", None, pattern3_1, pattern3_2, pattern3_3, pattern3_4, pattern3_5, pattern3_6, pattern3_7, 
            pattern3_8, pattern3_9, pattern3_10, pattern3_11, pattern3_12, pattern3_13, pattern3_14, pattern3_15, 
            pattern3_16, pattern3_17)

pattern4_1 = [{"LOWER":"create"}]
pattern4_2 = [{"LOWER":"define"}]
pattern4_3 = [{"LOWER":"declare"}]
pattern4_4 = [{"LOWER":"set"}]
pattern4_5 = [{"LOWER":"print"}]
pattern4_6 = [{"LOWER":"display"}]
pattern4_7 = [{"LOWER":"show"}]
pattern4_8 = [{"LOWER":"get"}]
pattern4_9 = [{"LOWER":"input"}]
pattern4_10 = [{"LOWER":"save"}]
matcher.add("COMMAND", None, pattern4_1, pattern4_2, pattern4_3, pattern4_4, pattern4_5, pattern4_6, pattern4_7, 
            pattern4_8, pattern4_9, pattern4_10)


In [4]:
#Creating Training Data

TRAINING_DATA = []

# Create a Doc object for each text in TEXTS
for doc in nlp.pipe(fileData):
    
    #x = re.sub("=", " = ", doc)
    #doc = re.sub("  "," ", x)
    
    # Match on the doc and create a list of matched spans
    spans = [doc[start:end] for match_id, start, end in matcher(doc)]
    
    #Modified Start
    entity_label=[]
    entity_label = [(nlp.vocab.strings[match_id]) for match_id, start, end in matcher(doc)]
    #if entity_label!=[]:
    #    print (entity_label)
    entities = [(span.start_char, span.end_char, entity_label[index]) for index,span in enumerate(spans)]
    
    #for eachWord in entities:
    #    print("Entities: {}".format(eachWord))
    
    # Format the matches as a (doc.text, entities) tuple
    training_example = (doc.text, {"entities": entities})
    
    # Append the example to the training data
    TRAINING_DATA.append(training_example)
    #Modified End
    
    '''
    entities = [(span.start_char, span.end_char, "INITIALIZATION") for span in spans]
    
    for eachWord in entities:
        print("Entities: {}".format(eachWord))
        
    # Format the matches as a (doc.text, entities) tuple
    training_example = (doc.text, {"entities": entities})
    # Append the example to the training data
    TRAINING_DATA.append(training_example)
    '''
    

print(*TRAINING_DATA, sep="\n")

('define global variable\n', {'entities': [(0, 6, 'COMMAND'), (7, 13, 'SCOPE'), (14, 22, 'DATATYPE')]})
('create global variable\n', {'entities': [(0, 6, 'COMMAND'), (7, 13, 'SCOPE'), (14, 22, 'DATATYPE')]})
('define variable\n', {'entities': [(0, 6, 'COMMAND'), (7, 15, 'DATATYPE')]})
('create variable\n', {'entities': [(0, 6, 'COMMAND'), (7, 15, 'DATATYPE')]})
('set variable equal to 10\n', {'entities': [(0, 3, 'COMMAND'), (4, 12, 'DATATYPE')]})
('set variable i = 10\n', {'entities': [(0, 3, 'COMMAND'), (4, 12, 'DATATYPE')]})
('define variable i\n', {'entities': [(0, 6, 'COMMAND'), (7, 15, 'DATATYPE')]})
('create variable i\n', {'entities': [(0, 6, 'COMMAND'), (7, 15, 'DATATYPE')]})
('set variable equal I to 0\n', {'entities': [(0, 3, 'COMMAND'), (4, 12, 'DATATYPE')]})
('set variable i = 0\n', {'entities': [(0, 3, 'COMMAND'), (4, 12, 'DATATYPE')]})
('define variable j\n', {'entities': [(0, 6, 'COMMAND'), (7, 15, 'DATATYPE')]})
('create variable j\n', {'entities': [(0, 6, 'COMMAND'), (

In [5]:
#Training Loop

import random

nlp = spacy.blank("en")
ner = nlp.create_pipe("ner")
nlp.add_pipe(ner)

ner.add_label("DATATYPE")
ner.add_label("SCOPE")
ner.add_label("OPERATOR")
ner.add_label("COMMAND")

# Start the training
nlp.begin_training()

# Loop for 10 iterations
for itn in range(10):
    # Shuffle the training data
    random.shuffle(TRAINING_DATA)
    losses = {}

    # Batch the examples and iterate over them
    for batch in spacy.util.minibatch(TRAINING_DATA, size=2):
        texts = [text for text, entities in batch]
        annotations = [entities for text, entities in batch]

        # Update the model
        nlp.update(texts, annotations, losses=losses)
    print(losses)

D:\Applications\miniconda3\envs\ME-NLP\lib\site-packages\spacy\language.py:635: UserWarning: [W033] Training a new parser or NER using a model with an empty lexeme normalization table. This may degrade the performance to some degree. If this is intentional or this language doesn't have a normalization table, please ignore this warning.
  proc.begin_training(
D:\Applications\miniconda3\envs\ME-NLP\lib\site-packages\spacy\language.py:635: UserWarning: [W034] Please install the package spacy-lookups-data in order to include the default lexeme normalization table for the language 'en'.
  proc.begin_training(


{'ner': 99.80915988740162}
{'ner': 5.736231205333718}
{'ner': 4.804834516031368}
{'ner': 3.7330646932041502}
{'ner': 1.1416002361990636}
{'ner': 0.00012271577004626836}
{'ner': 1.9693003599639788e-05}
{'ner': 5.413556165010714e-07}
{'ner': 2.3355843968861577e-07}
{'ner': 1.530519153528245e-07}


In [6]:
from spacy import displacy

doc = nlp("Print variable i to screen")
for ent in doc.ents:
    print (ent.text, ent.start_char, ent.end_char, ent.label_)

displacy.render(nlp(doc.text), style="ent", jupyter=True)

Print 0 5 COMMAND
variable 6 14 DATATYPE


In [7]:
doc = nlp("add numbers a and b")
for ent in doc.ents:
    print (ent.text, ent.start_char, ent.end_char, ent.label_)

displacy.render(nlp(doc.text), style="ent", jupyter=True)

add 0 3 COMMAND
